# 📊 Анализ рецептов терразитовой штукатурки

Этот ноутбук анализирует базу рецептов терразитовой штукатурки, используя унифицированные компоненты проекта.

## 1. Импорт библиотек и настройка окружения

In [ ]:
import sys
import os
from pathlib import Path

# Добавляем корневую директорию проекта в путь для импорта модулей
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Импортируем компоненты проекта
from src.data.loader import RecipeLoader, DataLoader
from src.utils.logger import setup_logger

# Настройка стилей графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Настройка логгера
logger = setup_logger('notebook_analysis')
print("✅ Библиотеки загружены")

## 2. Загрузка данных с использованием унифицированного RecipeLoader

In [ ]:
# Путь к Excel файлу
excel_path = Path('../data/raw/Рецептуры терразит.xlsx')

if not excel_path.exists():
    print(f"❌ Файл не найден: {excel_path}")
    print("📂 Убедитесь, что файл находится в data/raw/")
    print("📂 Или создайте тестовый файл с помощью create_test_excel.py")
else:
    print(f"✅ Файл найден: {excel_path.name}")
    print(f"📏 Размер файла: {excel_path.stat().st_size / 1024:.1f} KB")

In [ ]:
# Используем унифицированный RecipeLoader для загрузки данных
try:
    recipe_loader = RecipeLoader(excel_path)
    df = recipe_loader.load_excel()
    components = recipe_loader.parse_components()
    
    print(f"✅ Данные успешно загружены с использованием RecipeLoader")
    print(f"📊 Размер таблицы: {df.shape[0]} строк, {df.shape[1]} столбцов")
    print(f"🔧 Распарсено рецептов: {len(components)}")
    
    # Показать информацию о DataFrame
    print("\n📋 Информация о DataFrame:")
    print(f"Колонки ({len(df.columns)}): {list(df.columns)}")
    
    # Показать первые 3 строки
    print("\nПервые 3 строки данных:")
    display(df.head(3))
    
except Exception as e:
    print(f"❌ Ошибка при загрузке данных: {e}")
    import traceback
    traceback.print_exc()

## 3. Анализ структуры данных

In [ ]:
if 'components' in locals():
    # Анализ структуры компонентов
    print("📊 АНАЛИЗ СТРУКТУРЫ РЕЦЕПТОВ:")
    print("=" * 50)
    
    # 1. Общее количество рецептов
    print(f"1. Всего рецептов: {len(components)}")
    
    # 2. Уникальные компоненты
    all_components = set()
    for comp_dict in components.values():
        all_components.update(comp_dict.keys())
    print(f"2. Уникальных компонентов: {len(all_components)}")
    
    # 3. Компоненты по типам (если есть колонка 'Тип')
    if 'Тип' in df.columns:
        print(f"3. Типы рецептов:\n{df['Тип'].value_counts().to_string()}")
    
    # 4. Анализ названий рецептов
    if 'Название' in df.columns:
        print(f"4. Примеры названий рецептов:")
        for name in df['Название'].head(5):
            print(f"   • {name}")
    
    # 5. Статистика по компонентам
    component_stats = []
    for component in sorted(list(all_components))[:10]:  # Первые 10
        total = 0
        count = 0
        for comp_dict in components.values():
            if component in comp_dict:
                total += comp_dict[component]
                count += 1
        component_stats.append({
            'Компонент': component,
            'Используется в рецептах': count,
            'Среднее количество': total / count if count > 0 else 0,
            'Общее количество': total
        })
    
    print("5. Статистика по компонентам (топ-10):")
    stats_df = pd.DataFrame(component_stats)
    display(stats_df)


## 4. Сохранение обработанных данных в унифицированный формат

In [ ]:
# Сохранение в унифицированный JSON формат
if 'recipe_loader' in locals():
    try:
        # Используем метод RecipeLoader для сохранения
        output_path = Path('../data/processed/recipes.json')
        recipe_loader.save_to_json(output_path)
        
        print(f"✅ Данные сохранены в унифицированном формате: {output_path}")
        
        # Проверяем, что файл создан
        if output_path.exists():
            import json
            with open(output_path, 'r', encoding='utf-8') as f:
                saved_data = json.load(f)
            print(f"📊 Сохранено рецептов: {len(saved_data)}")
            print(f"📦 Размер файла: {output_path.stat().st_size / 1024:.1f} KB")
            
            # Показываем пример сохраненного рецепта
            if saved_data:
                print("\n📝 Пример сохраненного рецепта:")
                first_recipe = saved_data[0]
                print(f"   ID: {first_recipe.get('id')}")
                print(f"   Название: {first_recipe.get('name')}")
                print(f"   Тип: {first_recipe.get('type')}")
                print(f"   Компонентов: {len(first_recipe.get('components', {}))}")
        
    except Exception as e:
        print(f"❌ Ошибка при сохранении данных: {e}")

## 5. Визуализация данных

In [ ]:
if 'components' in locals() and len(components) > 0:
    # Создаем графики
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Распределение по типам (если есть)
    if 'Тип' in df.columns:
        type_counts = df['Тип'].value_counts()
        axes[0, 0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
        axes[0, 0].set_title('Распределение рецептов по типам')
    else:
        axes[0, 0].text(0.5, 0.5, 'Нет данных о типах', 
                       ha='center', va='center', fontsize=12)
        axes[0, 0].set_title('Распределение по типам')
    
    # 2. Количество компонентов в рецептах
    component_counts = [len(comp_dict) for comp_dict in components.values()]
    axes[0, 1].hist(component_counts, bins=15, edgecolor='black', alpha=0.7)
    axes[0, 1].set_xlabel('Количество компонентов')
    axes[0, 1].set_ylabel('Количество рецептов')
    axes[0, 1].set_title('Распределение количества компонентов')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Топ-10 компонентов по частоте использования
    component_freq = {}
    for comp_dict in components.values():
        for component in comp_dict.keys():
            component_freq[component] = component_freq.get(component, 0) + 1
    
    top_components = sorted(component_freq.items(), key=lambda x: x[1], reverse=True)[:10]
    component_names = [c[0][:20] + '...' if len(c[0]) > 20 else c[0] 
                       for c in top_components]
    frequencies = [c[1] for c in top_components]
    
    axes[1, 0].barh(range(len(component_names)), frequencies)
    axes[1, 0].set_yticks(range(len(component_names)))
    axes[1, 0].set_yticklabels(component_names)
    axes[1, 0].set_xlabel('Количество рецептов')
    axes[1, 0].set_title('Топ-10 компонентов по частоте использования')
    axes[1, 0].invert_yaxis()
    
    # 4. Общее количество компонентов
    axes[1, 1].text(0.5, 0.7, f'Всего рецептов: {len(components)}', 
                    ha='center', va='center', fontsize=14)
    axes[1, 1].text(0.5, 0.5, f'Уникальных компонентов: {len(all_components)}', 
                    ha='center', va='center', fontsize=14)
    axes[1, 1].text(0.5, 0.3, f'Среднее компонентов в рецепте: {np.mean(component_counts):.1f}', 
                    ha='center', va='center', fontsize=14)
    axes[1, 1].axis('off')
    axes[1, 1].set_title('Сводная статистика')
    
    plt.tight_layout()
    plt.show()
    
    # Сохраняем график
    output_dir = Path('../data/processed/figures')
    output_dir.mkdir(exist_ok=True)
    plt.savefig(output_dir / 'recipe_analysis.png', dpi=150, bbox_inches='tight')
    print(f"📈 Графики сохранены: {output_dir / 'recipe_analysis.png'}")

## 6. Детальный анализ компонентов

In [ ]:
# Анализ пигментов и других специфичных компонентов
if 'components' in locals():
    # Поиск компонентов, содержащих ключевые слова
    keywords = {
        'пигмент': [],
        'цемент': [],
        'песок': [],
        'мрамор': [],
        'кварц': [],
        'добавк': []
    }
    
    for component in all_components:
        component_lower = component.lower()
        for keyword in keywords:
            if keyword in component_lower:
                keywords[keyword].append(component)
    
    print("🔍 КЛАССИФИКАЦИЯ КОМПОНЕНТОВ:")
    print("=" * 50)
    
    for category, items in keywords.items():
        if items:
            print(f"{category.upper()} ({len(items)}):")
            for item in sorted(items)[:5]:  # Показываем первые 5
                print(f"  • {item}")
            if len(items) > 5:
                print(f"  ... и еще {len(items) - 5}")
            print()

## 7. Проверка совместимости с ML пайплайном

In [ ]:
# Проверяем, что сохраненные данные могут быть загружены DataLoader
json_path = Path('../data/processed/recipes.json')

if json_path.exists():
    try:
        # Используем DataLoader для загрузки сохраненных данных
        data_loader = DataLoader(json_path)
        recipes = data_loader.load_recipes()
        df_ml = data_loader.prepare_dataframe()
        
        print("✅ Проверка совместимости с ML пайплайном:")
        print(f"   • Загружено рецептов: {len(recipes)}")
        print(f"   • Создан DataFrame: {df_ml.shape}")
        print(f"   • Колонки: {df_ml.columns.tolist()[:5]}..." if len(df_ml.columns) > 5 else f"   • Колонки: {df_ml.columns.tolist()}")
        
        # Проверка возможности разделения данных
        try:
            X_train, X_test, y_train, y_test = data_loader.split_data()
            print(f"   • Данные разделены: train={X_train.shape[0]}, test={X_test.shape[0]}")
            print("🎉 Все проверки пройдены успешно!")
        except Exception as e:
            print(f"⚠️  Ошибка при разделении данных: {e}")
            print("   (Это нормально, если данные недостаточны для разделения)")
            
    except Exception as e:
        print(f"❌ Ошибка при проверке совместимости: {e}")
else:
    print("⚠️  JSON файл не найден для проверки совместимости")

## 8. Выводы и рекомендации

In [ ]:
print("=" * 60)
print("📊 ИТОГОВЫЙ ОТЧЕТ ПО АНАЛИЗУ ДАННЫХ")
print("=" * 60)

if 'components' in locals():
    print(f"\n📈 ОСНОВНЫЕ МЕТРИКИ:")
    print(f"• Всего рецептов: {len(components)}")
    print(f"• Уникальных компонентов: {len(all_components)}")
    
    if 'Тип' in df.columns:
        print(f"• Распределение по типам:")
        for typ, count in df['Тип'].value_counts().items():
            print(f"  - {typ}: {count} рецептов")
    
    # Статистика по компонентам
    component_counts = [len(comp_dict) for comp_dict in components.values()]
    print(f"\n📦 СТАТИСТИКА КОМПОНЕНТОВ:")
    print(f"• Среднее количество компонентов: {np.mean(component_counts):.1f}")
    print(f"• Минимальное количество: {np.min(component_counts)}")
    print(f"• Максимальное количество: {np.max(component_counts)}")
    
    print(f"\n🎯 РЕКОМЕНДАЦИИ ДЛЯ ML МОДЕЛИ:")
    print("1. Использовать унифицированный формат данных из recipes.json")
    print("2. Создать отдельные признаки для пигментов и заполнителей")
    print("3. Нормализовать данные по общему весу (1000 кг)")
    print("4. Рассмотреть возможность классификации по типам рецептов")
    
    print(f"\n📁 РЕЗУЛЬТАТЫ СОХРАНЕНЫ В:")
    print(f"• data/processed/recipes.json - унифицированные данные")
    print(f"• data/processed/figures/ - графики анализа")
    
    print(f"\n🚀 ДАЛЬНЕЙШИЕ ШАГИ:")
    print("1. Собрать фотографии образцов для каждого рецепта")
    print("2. Создать data_manifest.csv (связь фото-рецепт)")
    print("3. Запустить обучение модели на реальных данных")

else:
    print("\n❌ Данные не загружены. Проверьте наличие Excel файла.")